In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

# Verfying Kagge GPU and device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Using device:", torch.cuda.get_device_name(0))

# Use flan-t5-base for model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)


data = {
    "input_text": [
        "Net income increased by 15% year-over-year, driven by growth in subscription revenues.",
        "Operating expenses rose significantly due to increased investments in research and development.",
        "The company's debt-to-equity ratio stands at 2.5, indicating a highly leveraged position.",
        "Cash flow from operations was robust, exceeding $100 million for the quarter.",
        "Inventory turnover decreased by 10%, suggesting potential issues with demand or inventory management.",
        "The company's gross profit margin improved to 65% due to efficient cost control measures.",
        "Selling, general, and administrative expenses (SG&A) were reduced by 5% compared to the previous year.",
        "The effective tax rate for the year was 25%, resulting in a tax expense of $50 million.",
        "Capital expenditures totaled $75 million, primarily for expansion of production facilities.",
        "The board of directors approved a share repurchase program of $200 million.",
        "Despite a challenging market environment, revenue remained stable, demonstrating resilience.",
        "The company's current ratio is below 1.0, suggesting potential liquidity concerns.",
        "Research and development (R&D) spending increased by 20% to support innovation.",
        "The company recognized a one-time gain of $25 million from the sale of a non-core asset.",
        "Although revenue increased, net profit decreased due to foreign exchange losses.",
        "The company issued new bonds with a coupon rate of 6% to raise capital for acquisitions.",
        "The audit committee identified material weaknesses in the internal controls over financial reporting.",
        "The company's CEO received a significantly larger compensation package compared to the previous year.",
        "The company has been named as a co-defendant in a major intellectual property lawsuit.",
        "Auditors raise concerns about the company's related-party transactions.",
        
        "Reliance Industries Limited reported consolidated revenue of ₹968,702 crore in FY24, with EBITDA of ₹178,677 crore, demonstrating a 9.2% year-on-year growth.",
        "Tata Consultancy Services achieved ₹240,893 crore in revenue for FY24, with net margins at 18.6% and an operating cash flow to net income ratio of 111.4%.",
        "HDFC Bank's gross advances grew by 22.4% YoY to reach ₹25.07 lakh crore, with net interest margin stable at 3.45% as of Q4 FY24.",
        "ICICI Bank's CASA ratio improved to 44.6% in FY24 from 42.8% in FY23, while provisions decreased by 52.1% YoY to ₹1,049 crore in Q4 FY24.",
        "Infosys reported an EBIT margin of 20.7% in FY24, with free cash flow conversion at 88.6% of net profit and a capital allocation policy returning 85% of free cash flow.",
        "Bharti Airtel's ARPUs increased to ₹209 in Q4 FY24 from ₹178 in Q4 FY23, with data usage per customer reaching 21.8 GB per month.",
        "HCL Technologies announced revenue of ₹113,667 crore for FY24, with an EBITDA margin of 23.5% and diluted EPS of ₹46.75.",
        "Adani Enterprises reported consolidated EBITDA of ₹20,321 crore in FY24, up 45.6% YoY, with capital expenditure of ₹33,660 crore primarily in airport and road infrastructure.",
        "State Bank of India's gross NPA ratio improved to 2.21% in Q4 FY24 from 2.78% in Q4 FY23, while the net NPA ratio declined to 0.58% from 0.67%.",
        "Larsen & Toubro secured new orders worth ₹323,160 crore in FY24, representing a book-to-bill ratio of 1.3x and international orders constituting 35% of the total.",
        "Wipro's IT Services segment generated revenue of ₹89,687 crore in FY24, with voluntary attrition decreasing to 14.2% from 19.4% in the previous fiscal year.",
        "ITC Limited reported a 10.2% growth in FMCG-Others segment revenue to ₹19,366 crore in FY24, with EBITDA margin expanding by 150 basis points to 12.3%.",
        "Mahindra & Mahindra's automotive segment witnessed a volume growth of 18.7% YoY in FY24, with a significant 47.5% increase in SUV volumes and EBITDA margin of 14.8%.",
        "Asian Paints recorded consolidated revenue of ₹37,723 crore in FY24, with decorative business volume growth of 11.5% and gross margin expansion of 320 basis points YoY.",
        "Coal India Limited achieved coal production of 773.6 million tonnes in FY24, up 10.2% YoY, with e-auction premiums averaging 82% above notified prices.",
        
        "What is EBITDA?",
        "What is Net Interest Margin?",
        "What is a Gross NPA ratio?",
        "What is Capital Adequacy Ratio?",
        "What is Return on Capital Employed (ROCE)?",
    ],
    "target_text": [
        "The company's profits grew 15% thanks to more people subscribing to its services.",
        "The company spent more money on developing new products and services, causing operating costs to increase.",
        "The company has a lot of debt compared to its ownership, which is a high-risk situation.",
        "The company generated a large amount of cash from its business operations this quarter.",
        "The company is selling its products more slowly, indicating possible problems with sales or how well it's managing its supplies.",
        "The company made more profit from each sale by controlling its costs effectively.",
        "The company spent less money on sales, marketing, and general administration compared to last year.",
        "The company paid $50 million in taxes this year, with an overall tax rate of 25%.",
        "The company spent $75 million on new facilities to increase its production capacity.",
        "The company will buy back its own shares worth $200 million.",
        "Even though the market was tough, the company managed to maintain its sales level.",
        "The company might have short-term cash problems because it owes more money than it has readily available.",
        "The company is investing more in new ideas and technologies.",
        "The company sold one of its less important assets and made $25 million from it.",
        "Even though sales were up, the company made less money because it lost money on currency exchange.",
        "The company borrowed money at a 6% interest rate, planning to use it for buying other businesses.",
        "The company has problems with how it keeps track of its money, as pointed out by the auditors.",
        "The CEO received a much bigger paycheck this year than last year.",
        "The company is facing a major lawsuit over intellectual property rights.",
        "The auditors are worried about deals the company made with its related parties.",
        
        "Reliance made ₹968,702 crore in sales this year, with their operating profit growing by 9.2% compared to last year.",
        "TCS earned ₹240,893 crore this year, keeping 18.6% of that as profit and generating more cash than its reported profit.",
        "HDFC Bank increased its loans by 22.4% compared to last year, reaching ₹25.07 lakh crore, while maintaining stable profit margins on these loans.",
        "ICICI Bank improved the amount of low-cost deposits it holds, while needing to set aside much less money for bad loans compared to last year.",
        "Infosys maintained a healthy 20.7% profit margin and is giving back 85% of its extra cash to shareholders.",
        "Bharti Airtel is earning more money per customer (₹209 compared to ₹178 last year), with customers using more data than before.",
        "HCL Technologies earned ₹113,667 crore this year, with about 23.5% of that being profit before taxes and interest payments.",
        "Adani Enterprises' operating profit grew by 45.6% compared to last year, with the company investing heavily in airports and roads.",
        "State Bank of India has improved the quality of its loans, with fewer loans going bad compared to last year.",
        "Larsen & Toubro won ₹323,160 crore worth of new projects this year, with 35% of these coming from international clients.",
        "Wipro's main IT business earned ₹89,687 crore, with fewer employees leaving the company compared to last year.",
        "ITC's consumer products segment (excluding cigarettes) grew by 10.2%, with profit margins improving significantly.",
        "Mahindra & Mahindra sold 18.7% more vehicles than last year, with SUV sales growing by 47.5% and healthy profit margins.",
        "Asian Paints earned ₹37,723 crore this year, selling 11.5% more paint and making more profit on each sale.",
        "Coal India produced 773.6 million tonnes of coal this year (10.2% more than last year) and sold coal in auctions at prices 82% higher than standard rates.",
        
        # Simplified explanations for financial terms
        "EBITDA is a company's earnings before deducting interest, taxes, depreciation, and amortization - it shows how much money a company makes from its core operations.",
        "Net Interest Margin is the difference between what a bank earns from loans and what it pays for deposits, shown as a percentage - higher is usually better.",
        "Gross NPA ratio shows what percentage of a bank's loans aren't being paid back on time - a lower number is healthier.",
        "Capital Adequacy Ratio is how much money a bank has compared to its risky assets - it's a safety cushion against losses.",
        "ROCE measures how efficiently a company uses its capital to generate profits - it tells you how many rupees of profit are made for each rupee invested in the business.",
    ]
}

# Create a Hugging Face dataset and split into train and evaluation sets
dataset = Dataset.from_dict(data)
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

def tokenize_function(example):
    model_inputs = tokenizer(example["input_text"], max_length=512, truncation=True)
    labels = tokenizer(example["target_text"], max_length=150, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenizing DataSets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

# Set up training arguments (using fp16 for T4)
training_args = TrainingArguments(
    output_dir="./flan_t5_indian_finance",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-5,
    fp16=True,
    dataloader_num_workers=2,
    report_to=[],
    gradient_accumulation_steps=2
)

# Initialize Data Collator for Seq2Seq tasks to handle dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)

# Start fine-tuning
trainer.train()

# Save the model
model.save_pretrained("./flan_t5_indian_finance")
tokenizer.save_pretrained("./flan_t5_indian_finance")

GPU available: True
Using device: Tesla T4


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.180422
2,No log,2.132757
3,No log,2.111821


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


('./flan_t5_indian_finance/tokenizer_config.json',
 './flan_t5_indian_finance/special_tokens_map.json',
 './flan_t5_indian_finance/spiece.model',
 './flan_t5_indian_finance/added_tokens.json',
 './flan_t5_indian_finance/tokenizer.json')

# Inferenece code

In [4]:

model_path = "./flan_t5_indian_finance"
try:
    print("Loadng fine-tuned model from", model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
except Exception as e:
    print(f"Error loading fine-tuned model: {e}")
    print("Fall through to base model, for APIs and NGRock")
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.eval()

def generate_simplified_explanation(financial_text):
    """
    In the given Financial Reports , I want you to Simplify those financial reports in form of a story and for 15 year old, Simplify technical terms into easiers 
    terms.
    """
   
    prompt = f"simplify this financial statement: {financial_text}"
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=512,
        padding="max_length",
        truncation=True
    ).to(device)
    
    
    with torch.no_grad():  
        outputs = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=150,
            min_length=20,  # Prevent very short outputs like "ssss"
            do_sample=True,  # Enable sampling for more diverse outputs
            temperature=0.7,  # Control randomness (lower = more deterministic)
            top_p=0.9,        # Nucleus sampling
            num_beams=4,      # Beam search for better quality
            early_stopping=True
        )
    
    # Output sampling 
    simplified_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
 # Fall back situation
    if not simplified_text or simplified_text.isspace() or "s" * 3 in simplified_text:
        return "The model failed to produce a meaningful simplification. This financial statement discusses " + \
               financial_text.split()[0:5] + "... Please try with a different statement or model."
    
    return simplified_text

complex_reports = [
    # Example 1: Tata Steel
    """
    Tata Steel Limited reported consolidated revenues of ₹223,519 crore for FY24, a decline of 7.2% YoY primarily 
    due to lower realizations across key markets. EBITDA stood at ₹25,323 crore, with margins contracting by 380 
    basis points to 11.3%. The company's consolidated PAT declined by 53.4% to ₹8,356 crore.
    """,
    
    # Example 2: HDFC Bank
    """
    HDFC Bank's gross advances grew by 22.4% YoY to reach ₹25.07 lakh crore, with net interest margin stable 
    at 3.45% as of Q4 FY24. The bank reported a Capital Adequacy Ratio of 19.2%, while maintaining a Gross NPA 
    ratio of 1.24%, reflecting strong asset quality management in a challenging economic environment.
    """,
    
    # Example 3: Simple financial term explanation
    "What is EBITDA and why is it important for evaluating a company's financial performance?",
    
    # Example 4: Short statement for testing
    "The company's current ratio is below 1.0, suggesting potential liquidity concerns.",
]

# Test with each example
print("===== FINANCIAL STATEMENT SIMPLIFICATION =====\n")
for i, report in enumerate(complex_reports):
    print(f"\n----- COMPLEX REPORT #{i+1} -----")
    print(report)
    print("\n----- SIMPLIFIED EXPLANATION -----")
    simplified = generate_simplified_explanation(report)
    print(simplified)
    print("\n" + "="*50)

# Test with a longer, more complex report
complex_report = """
Bajaj Finance Limited reported a 33% YoY growth in Assets Under Management (AUM) to ₹325,523 crore in FY24, 
with the consolidated customer franchise expanding to 85.6 million (up 20% YoY). Net Interest Income (NII) 
grew by 28% to ₹33,411 crore, while Net Interest Margin (NIM) contracted by 25 bps to 10.2%. The Gross NPA 
ratio and Net NPA ratio improved to 0.85% and 0.28% respectively, compared to 0.94% and 0.34% in FY23. The 
company's Capital Adequacy Ratio (including Tier-II capital) stood at a robust 22.55%, with a Tier-I capital 
ratio of 20.01%. The provisioning coverage ratio improved to 67%, while the liquidity buffer was maintained 
at 9.1% of total borrowings.
"""

print("\n\n===== DETAILED EXAMPLE =====")
print("\nOriginal Financial Report:")
print(complex_report)
print("\nSimplified Explanation:")
simplified = generate_simplified_explanation(complex_report)
print(simplified)

Loading fine-tuned model from ./flan_t5_indian_finance
===== FINANCIAL STATEMENT SIMPLIFICATION =====


----- COMPLEX REPORT #1 -----

    Tata Steel Limited reported consolidated revenues of ₹223,519 crore for FY24, a decline of 7.2% YoY primarily 
    due to lower realizations across key markets. EBITDA stood at ₹25,323 crore, with margins contracting by 380 
    basis points to 11.3%. The company's consolidated PAT declined by 53.4% to ₹8,356 crore.
    

----- SIMPLIFIED EXPLANATION -----
Tata Steel Limited reported consolidated revenues of 223,519 crore for FY24, a decline of 7.2% YoY primarily due to lower realizations across key markets.


----- COMPLEX REPORT #2 -----

    HDFC Bank's gross advances grew by 22.4% YoY to reach ₹25.07 lakh crore, with net interest margin stable 
    at 3.45% as of Q4 FY24. The bank reported a Capital Adequacy Ratio of 19.2%, while maintaining a Gross NPA 
    ratio of 1.24%, reflecting strong asset quality management in a challenging economic env